In [1]:
import pickle
import numpy as np
import pandas as pd

# Part 1: Read data

In [2]:
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
customer_data = load_obj("data/customer_data")

In [4]:
customer_data.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

# Part 2 Generate Rating dataframe <br>
#### Customer's holding could be taked as a rating to one stock, for eg, if customer is holding 70% of position for apple, we could think his raing for apple is 0.7

In [5]:
### Loop over data to generate rating dataframe
ratingList = []
for customer in customer_data.keys():
    tmp_data = customer_data[customer]
    for stock, rating in zip(tmp_data['stocks'],tmp_data['positions']):
        ratingList.append([customer,stock,rating])

In [6]:
df_rating = pd.DataFrame(ratingList,columns=["customerID","stockID","raing"])
df_rating.head()

,customerID,stockID,raing
0,0,XRX,0.326066
1,0,MDLZ,0.554392
2,0,BSX,0.119542
3,1,OMC,0.046653
4,1,JBHT,0.826472


In [7]:
stock2id = {}
for id,stock in enumerate(df_rating.stockID.unique()):
    stock2id[stock] = id

In [8]:
df_rating.stockID = df_rating.stockID.apply(lambda x:stock2id[x])

In [9]:
df_rating.head()

,customerID,stockID,raing
0,0,0,0.326066
1,0,1,0.554392
2,0,2,0.119542
3,1,3,0.046653
4,1,4,0.826472


# Part 3: Collaborative filtering (ALS) with Spark

In [10]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.mllib.recommendation import ALS,MatrixFactorizationModel, Rating


In [11]:
"""Create SparkContext"""
### You can only have one sparkContext at the same tim
sc = pyspark.SparkContext.getOrCreate()

sqlContext = SQLContext(sc)

In [12]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [16]:
"""Create Spark dataframe"""
df_spark = sqlContext.createDataFrame(df_rating)
df_spark.printSchema()

root
 |-- customerID: long (nullable = true)
 |-- stockID: long (nullable = true)
 |-- raing: double (nullable = true)



In [17]:
df_spark.head(5)

Py4JJavaError: An error occurred while calling o154.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2, localhost, executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:170)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:97)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:108)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:324)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:288)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint

In [15]:
"""Train ALS model """
rank = 5
numIterations = 10
model = ALS.train(df_spark, rank, numIterations) ### link to ALS model parameters explaination https://jaceklaskowski.gitbooks.io/mastering-apache-spark/spark-mllib/spark-mllib-ALS.html

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1, localhost, executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:170)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:97)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:108)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:324)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:288)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.R

In [41]:
"""Recommendation based on Users & Products"""
display('features for stock one', model.productFeatures().first())
display('features for user one',model.userFeatures().first())

'features for stock one'

(0,
 array('d', [0.12633837759494781, 0.12388911843299866, -0.015953101217746735, 0.556143045425415, 0.15344947576522827]))

'features for user one'

(0,
 array('d', [-0.16528134047985077, -0.20782460272312164, 0.367808073759079, 0.3977356553077698, 0.8150374889373779]))

In [42]:
# For Product X, Find N Users to Sell To
stockForUser = model.recommendUsers(242,10)
print("Top 10 stock for user 242: \n")
display(stockForUser)

Top 10 stock for user 242: 



[Rating(user=452, product=242, rating=0.3602184931050588),
 Rating(user=67, product=242, rating=0.3463501991220613),
 Rating(user=155, product=242, rating=0.34515986881657057),
 Rating(user=501, product=242, rating=0.34293597652068003),
 Rating(user=123, product=242, rating=0.33604757132732427),
 Rating(user=992, product=242, rating=0.33181805737198466),
 Rating(user=628, product=242, rating=0.3291330899171424),
 Rating(user=144, product=242, rating=0.32782366856595346),
 Rating(user=252, product=242, rating=0.3216203492535704),
 Rating(user=190, product=242, rating=0.3205132401075211)]

In [43]:
# For User Y Find N Products to Promote
userForStock = model.recommendProducts(196,10)
print("Top 10 users to recommend for stock 196")
display(userForStock)

Top 10 users to recommend for stock 196


[Rating(user=196, product=125, rating=0.39763502590488475),
 Rating(user=196, product=500, rating=0.38190978210304416),
 Rating(user=196, product=298, rating=0.38028967421617105),
 Rating(user=196, product=390, rating=0.37875007147166206),
 Rating(user=196, product=452, rating=0.37064918141782255),
 Rating(user=196, product=81, rating=0.36102658692014367),
 Rating(user=196, product=207, rating=0.35912423743422395),
 Rating(user=196, product=454, rating=0.3380689060138194),
 Rating(user=196, product=258, rating=0.33379328036881617),
 Rating(user=196, product=153, rating=0.32174814249281747)]

In [44]:
# Predict ratings
ratingPredict= model.predict(196,242)
print("prediction of rate for user 196 --> stock 242")
display(ratingPredict)

prediction of rate for user 196 --> stock 242


0.10230535586244738

In [45]:
stockFeatures = model.productFeatures()
stockFeatures.collect()

[(0,
  array('d', [0.12633837759494781, 0.12388911843299866, -0.015953101217746735, 0.556143045425415, 0.15344947576522827])),
 (8,
  array('d', [0.22152769565582275, 0.38128241896629333, 0.33698931336402893, 0.38808056712150574, -0.03439795598387718])),
 (16,
  array('d', [-0.2633485198020935, -0.27575016021728516, -0.02187899872660637, 0.3068324029445648, 0.2818983495235443])),
 (24,
  array('d', [0.09938246011734009, 0.322505921125412, -0.3264167606830597, 0.1840282529592514, -0.11190211027860641])),
 (32,
  array('d', [0.2880896031856537, -0.1458972543478012, 0.27591201663017273, 0.36329033970832825, 0.3246811330318451])),
 (40,
  array('d', [-0.22399356961250305, 0.47305282950401306, 0.03626835346221924, 0.1976516991853714, -0.04231593385338783])),
 (48,
  array('d', [0.29835158586502075, -0.009157358668744564, -0.30552244186401367, 0.02133513055741787, -0.10125067830085754])),
 (56,
  array('d', [0.18635356426239014, 0.6517741084098816, 0.2740646302700043, 0.18304647505283356, 0.

# Part 3: modulize

In [46]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.mllib.recommendation import ALS,MatrixFactorizationModel, Rating
import pandas as pd

class recommendor:

    def __init__(userIDs,productIDs,ratings,numIterations,rank=5,sc=None):
        if not sc:
            sc = pyspark.SparkContext.getOrCreate()

        self.sc = sc ### sc is spark session
        self.df = pd.DataFrame({"userIDs":userIDs,"productIDs":productIDs,'ratings':ratings})
        self.sqlContext = SQLContext(self.sc)
        self.df_spark = sqlContext.createDataFrame(self.df)
        self.rank = rank 
        self.numIterations = numIterations
        
        self.model = ALS.train(self.df_spark, self.rank,self.numIterations) 
                
    def rating_user_2_product(self,userID,productID):
        return self.model.predict(userID,productID)
    def get_user_features(self,userID):
        return self.model.userFeatures().collect()
    def get_product_features(self,productID):
        return self.model.productFeatures().collect()
    def recommend_users(userID,num_products=10):
        return self.model.recommendUsers(userID,num_products)
    def recommend_products(productID,num_users=10):
        return self.model.recommendProducts(productID,num_users)
    